In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

#### 1. Loading in our data

In [2]:
weather_df: pd.DataFrame = pd.read_feather("./training_data/processed_data/preprocessed_weather_hartbeespoort.feather")
images_df: pd.DataFrame = pd.read_feather("./training_data/processed_data/preprocessed_image_hartbeespoort.feather")

In [3]:
weather_df

,windspeed,winddir
datetime,,
2021-01-04,18.4,292.2
2021-01-09,15.4,330.5
2021-01-21,21.7,85.6
2021-02-10,13.6,46.0
2021-02-13,16.6,117.2
...,...,...
2024-05-10,14.8,320.4
2024-05-13,13.9,29.4
2024-05-15,16.4,183.5


In [4]:
images_df

,center_x_1,center_y_1,x_axis_length_1,y_axis_length_1,angle_1,center_x_2,center_y_2,x_axis_length_2,y_axis_length_2,angle_2,center_x_3,center_y_3,x_axis_length_3,y_axis_length_3,angle_3
datetime,,,,,,,,,,,,,,,
2021-01-04,704.028564,442.499176,121.826691,451.231567,58.160427,960.999756,336.523529,112.876732,203.835342,117.014908,1092.642456,257.937164,133.023499,147.132126,142.699448
2021-01-09,715.279602,351.457184,100.243324,367.353882,66.100555,807.207031,124.622177,54.882431,117.928635,119.937805,184.712097,572.285339,51.416313,93.895416,133.982315
2021-01-21,325.976746,490.977356,136.785980,500.891998,74.853523,813.122131,146.077484,87.973320,149.937958,163.590485,1233.187134,566.028442,8.314092,24.543709,143.964798
2021-02-10,167.179871,545.132202,91.021637,191.275513,108.308769,881.044556,144.769745,63.307846,291.755676,111.700424,592.362976,408.737030,60.006161,177.342026,67.738052
2021-02-13,280.800171,504.206818,134.417603,405.863342,72.619293,787.453369,126.050583,35.755112,48.203991,129.081055,918.028381,556.878296,14.486405,27.316944,2.405948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-10,777.354004,107.631660,30.748890,90.560562,146.724838,243.170166,542.966003,13.301406,35.620926,173.263412,713.954590,443.647552,9.775264,37.554825,63.395508
2024-05-13,716.543701,305.779480,22.611246,165.610962,61.839939,120.704819,541.091553,13.267200,20.426884,71.220276,797.884949,141.748245,10.134691,28.270313,174.367538
2024-05-15,800.668274,145.144821,26.662622,81.719994,131.096573,756.216187,80.918289,15.954613,37.925011,145.252411,112.768906,556.176453,10.941070,24.187414,82.034576


#### 2. Preparing our data

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

result: list[pd.DataFrame] = train_test_split(weather_df, images_df, test_size=0.1, random_state=69, shuffle=False)
X_train: pd.DataFrame = result[0]
X_valid: pd.DataFrame = result[1]
y_train: pd.DataFrame = result[2]
y_valid: pd.DataFrame = result[3]

print(f"Training data rows: {len(X_train)}, Test data rows: {len(X_valid)}")

Training data rows: 287, Test data rows: 32


In [6]:
def print_metrics(actual, prediction, data_type) -> None:
    print("Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)")
    print(f"{data_type} DATA: MSE: {mean_squared_error(actual, prediction)}")
    print(f"{data_type} DATA: RMSE: {mean_squared_error(actual, prediction, squared=False)}")
    print(f"{data_type} DATA: MAE: {mean_absolute_error(actual, prediction)}\n")

#### 3. Testing different types of supervised regression models

##### 3.1. Decision Tree Regressor

In [7]:
from sklearn.tree import DecisionTreeRegressor

# tree_model = DecisionTreeRegressor(min_samples_split=10, min_samples_leaf=10, min_weight_fraction_leaf=0.3)
tree_model = DecisionTreeRegressor()

tree_model.fit(X_train, y_train)

tree_y_pred: np.ndarray = tree_model.predict(X_train)
print_metrics(y_train, tree_y_pred, "KNOWN")

tree_y_pred: np.ndarray = tree_model.predict(X_valid)
print_metrics(y_valid, tree_y_pred, "NEW")

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 0.0
KNOWN DATA: RMSE: 0.0
KNOWN DATA: MAE: 0.0

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 68817.6590833848
NEW DATA: RMSE: 202.138128084863
NEW DATA: MAE: 163.8141901485622



##### 3.1.1. Use GridSearch for attempt at optimal hyperparameters

In [8]:
params_tree = {
    "criterion": ["squared_error", "friedman_mse", "absolute_error", "poisson"],
    "max_depth": [None, 2, 4, 10, 20],
    "min_samples_split": [2, 4, 5, 10],
    "min_samples_leaf": [1, 2, 4, 5, 10],
    "min_weight_fraction_leaf": [0.0, 0.1, 0.2, 0.3],
    "max_leaf_nodes": [None, 6, 8, 10, 20],
    "min_impurity_decrease": [0.0, 0.1, 0.2, 0.3]
}

grid_tree = GridSearchCV(tree_model, param_grid=params_tree, n_jobs=-1, verbose=True)

grid_tree.fit(X_train, y_train)

grid_tree.best_params_

Fitting 5 folds for each of 32000 candidates, totalling 160000 fits


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy

{'criterion': 'friedman_mse',
 'max_depth': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.2}

##### 3.1.2. Testing the newly acquired hyperparameters

In [8]:
tree_model = DecisionTreeRegressor(criterion="friedman_mse", min_weight_fraction_leaf=0.2)

tree_model.fit(X_train, y_train)

tree_y_pred: np.ndarray = tree_model.predict(X_train)
print_metrics(y_train, tree_y_pred, "KNOWN")

tree_y_pred: np.ndarray = tree_model.predict(X_valid)
print_metrics(y_valid, tree_y_pred, "NEW")

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 31528.223894258812
KNOWN DATA: RMSE: 138.47504761231784
KNOWN DATA: MAE: 112.24267252072096

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 32809.82534135598
NEW DATA: RMSE: 140.77275305614256
NEW DATA: MAE: 115.88433876852582



Here we can see that our hyperparameters are a good balance between good behaviour on the known values and the new values.

##### Non-default hyperparameters:

- criterion (default="squared_error"): The function to measure the quality of a split. 
- min_weight_fraction_leaf (default=0.0): The minimum weighted fraction of the sum total of weights (of all the input samples) required to be at a leaf node.


[scikit-learn DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

##### 3.2. Random Forest Regressor

In [9]:
from sklearn.ensemble import RandomForestRegressor

# forest_model = RandomForestRegressor(min_samples_leaf=10, min_weight_fraction_leaf=0.3)
forest_model = RandomForestRegressor()

forest_model.fit(X_train, y_train)

forest_y_pred: np.ndarray = forest_model.predict(X_train)
print_metrics(y_train, forest_y_pred, "KNOWN")

forest_y_pred: np.ndarray = forest_model.predict(X_valid)
print_metrics(y_valid, forest_y_pred, "NEW")

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 5200.383832797404
KNOWN DATA: RMSE: 55.93087375357399
KNOWN DATA: MAE: 44.47791441865194

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 42069.282464517244
NEW DATA: RMSE: 156.4201334639297
NEW DATA: MAE: 129.21796485854438



##### 3.2.1. Use GridSearch for attempt at optimal hyperparameters

In [10]:
params_forest = {
    "n_estimators": [10, 25, 50],
    "criterion": ["squared_error", "friedman_mse", "absolute_error", "poisson"],
    "max_depth": [None, 2, 4, 10],
    "min_samples_split": [2, 4, 5],
    "min_samples_leaf": [1, 2, 4, 5],
    "min_weight_fraction_leaf": [0.0, 0.1, 0.2, 0.3],
    "max_leaf_nodes": [None, 6, 8, 10],
    "min_impurity_decrease": [0.0, 0.1, 0.2, 0.3]
}

grid_forest = GridSearchCV(forest_model, param_grid=params_forest, n_jobs=-1, verbose=True)

grid_forest.fit(X_train, y_train)

grid_forest.best_params_

Fitting 5 folds for each of 36864 candidates, totalling 184320 fits


{'criterion': 'friedman_mse',
 'max_depth': 2,
 'max_leaf_nodes': 6,
 'min_impurity_decrease': 0.3,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'min_weight_fraction_leaf': 0.1,
 'n_estimators': 10}

##### 3.2.2. Testing the newly acquired hyperparameters

In [10]:
forest_model = RandomForestRegressor(n_estimators=10, criterion="friedman_mse", max_depth=2, max_leaf_nodes=6,
                                     min_impurity_decrease=0.3, min_samples_split=5, min_weight_fraction_leaf=0.1)

forest_model.fit(X_train, y_train)

forest_y_pred: np.ndarray = forest_model.predict(X_train)
print_metrics(y_train, forest_y_pred, "KNOWN")

forest_y_pred: np.ndarray = forest_model.predict(X_valid)
print_metrics(y_valid, forest_y_pred, "NEW")

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 31142.10614912715
KNOWN DATA: RMSE: 137.81212009619233
KNOWN DATA: MAE: 112.01805178523502

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 33202.4572309436
NEW DATA: RMSE: 141.91893981503964
NEW DATA: MAE: 116.91365470124158



Once again these parameters result in a good balance between train and test data performance.

##### Non-default hyperparameters:

- n_estimators (default=100): The number of trees in the forest.
- criterion (default="squared_error"): The function to measure the quality of a split.
- max_depth (default=None): The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
- min_samples_split (default=2): The minimum number of samples required to split an internal node.
- max_leaf_nodes (default=None): Grow trees with max_leaf_nodes in best-first fashion. Best nodes are defined as relative reduction in impurity. If None then unlimited number of leaf nodes.
- min_impurity_decrease (default=0.0): A node will be split if this split induces a decrease of the impurity greater than or equal to this value.
- min_weight_fraction_leaf (default=0.0): The minimum weighted fraction of the sum total of weights (of all the input samples) required to be at a leaf node.

[scikit-learn RandomForestRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

##### 3.3. Linear Regressor

In [11]:
from sklearn.linear_model import LinearRegression

# linear_model = LinearRegression(positive=True)
linear_model = LinearRegression()

linear_model.fit(X_train, y_train)

linear_y_pred: np.ndarray = linear_model.predict(X_train)
print_metrics(y_train, linear_y_pred, "KNOWN")

linear_y_pred: np.ndarray = linear_model.predict(X_valid)
print_metrics(y_valid, linear_y_pred, "NEW")

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 32201.504929329716
KNOWN DATA: RMSE: 139.7326368897543
KNOWN DATA: MAE: 113.8009497001413

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 33888.613503382105
NEW DATA: RMSE: 142.42283683389056
NEW DATA: MAE: 116.19241510558375



##### 3.3.1. Use GridSearch for attempt at optimal hyperparameters

In [12]:
params_linear = {
    "fit_intercept": [True, False],
    "positive": [False, True]
}

grid_linear = GridSearchCV(linear_model, param_grid=params_linear, n_jobs=-1, verbose=True)

grid_linear.fit(X_train, y_train)

grid_linear.best_params_

Fitting 5 folds for each of 4 candidates, totalling 20 fits


{'fit_intercept': True, 'positive': True}

##### 3.3.2. Testing the newly acquired hyperparameters

In [12]:
linear_model = LinearRegression(positive=True)

linear_model.fit(X_train, y_train)

linear_y_pred: np.ndarray = linear_model.predict(X_train)
print_metrics(y_train, linear_y_pred, "KNOWN")

linear_y_pred: np.ndarray = linear_model.predict(X_valid)
print_metrics(y_valid, linear_y_pred, "NEW")

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 32412.473880083835
KNOWN DATA: RMSE: 140.13840079242888
KNOWN DATA: MAE: 114.45904001584911

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 32201.900449802193
NEW DATA: RMSE: 139.62326538527105
NEW DATA: MAE: 114.78593845659337



Since there weren't many hyperparameters to tune this barely improved the model's performance.

##### Non-default hyperparameters:

- positive (default=False): When set to True, forces the coefficients to be positive.

[scikit-learn LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

##### 3.4. Ridge Regression

In [13]:
from sklearn.linear_model import Ridge

# ridge_model = Ridge(solver="lbfgs", positive=True)
ridge_model = Ridge()

ridge_model.fit(X_train, y_train)

ridge_y_pred: np.ndarray = ridge_model.predict(X_train)
print_metrics(y_train, ridge_y_pred, "KNOWN")

ridge_y_pred: np.ndarray = ridge_model.predict(X_valid)
print_metrics(y_valid, ridge_y_pred, "NEW")

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 32201.50493109628
KNOWN DATA: RMSE: 139.73263689419736
KNOWN DATA: MAE: 113.80098139933503

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 33888.47623139086
NEW DATA: RMSE: 142.42262954938604
NEW DATA: MAE: 116.19228669280068



##### 3.4.1. Use GridSearch for attempt at optimal hyperparameters

In [14]:
params_ridge = {
    "alpha": [1.0, 2.0, 2.5, 5.0, 10.0],
    "fit_intercept": [True, False],
    "max_iter": [None, 1000, 15000, 30000],
    "tol": [0.0001, 0.001, 0.01, 0.1, 0.00001],
    "solver": ["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga", "lbfgs"],
    "positive": [False, True]
}

grid_ridge = GridSearchCV(ridge_model, param_grid=params_ridge, n_jobs=-1, verbose=True)

grid_ridge.fit(X_train, y_train)

grid_ridge.best_params_

Fitting 5 folds for each of 3200 candidates, totalling 16000 fits


{'alpha': 10.0,
 'fit_intercept': True,
 'max_iter': None,
 'positive': True,
 'solver': 'auto',
 'tol': 0.1}

##### 3.4.2. Testing the newly acquired hyperparameters

In [14]:
ridge_model = Ridge(alpha=10.0, positive=True, tol=0.1)

ridge_model.fit(X_train, y_train)

ridge_y_pred: np.ndarray = ridge_model.predict(X_train)
print_metrics(y_train, ridge_y_pred, "KNOWN")

ridge_y_pred: np.ndarray = ridge_model.predict(X_valid)
print_metrics(y_valid, ridge_y_pred, "NEW")

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 32476.76812178059
KNOWN DATA: RMSE: 140.40212292213081
KNOWN DATA: MAE: 114.74507282422867

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 32431.311246263256
NEW DATA: RMSE: 140.4664504591828
NEW DATA: MAE: 115.54874335312698



We once again get fairly similar performance

##### Non-default hyperparameters:

- alpha (default=1.0): Constant that multiplies the L2 term, controlling regularization strength.
- positive (default=False): When set to True, forces the coefficients to be positive. Only ‘lbfgs’ solver is supported in this case.
- tol (default=0.0001): The precision of the solution (coef_) is determined by tol which specifies a different convergence criterion for each solver.

[scikit-learn Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)

##### 3.5. Lasso Regression

In [15]:
from sklearn.linear_model import Lasso

# lasso_model = Lasso(positive=True)
lasso_model = Lasso()

lasso_model.fit(X_train, y_train)

lasso_y_pred: np.ndarray = lasso_model.predict(X_train)
print_metrics(y_train, lasso_y_pred, "KNOWN")

lasso_y_pred: np.ndarray = lasso_model.predict(X_valid)
print_metrics(y_valid, lasso_y_pred, "NEW")

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 32201.526768166357
KNOWN DATA: RMSE: 139.73280033320987
KNOWN DATA: MAE: 113.80480170393795

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 33875.929759185085
NEW DATA: RMSE: 142.40251619534735
NEW DATA: MAE: 116.18344325520374



##### 3.5.1. Use GridSearch for attempt at optimal hyperparameters

In [16]:
params_lasso = {
    "alpha": [1.0, 2.0, 2.5, 5.0, 10.0],
    "fit_intercept": [True, False],
    "max_iter": [1000, 2000, 5000],
    "tol": [0.0001, 0.001, 0.01, 0.1],
    "positive": [False, True],
    "selection": ["cyclic", "random"]
}

grid_lasso = GridSearchCV(lasso_model, param_grid=params_lasso, n_jobs=-1, verbose=True)

grid_lasso.fit(X_train, y_train)

grid_lasso.best_params_

Fitting 5 folds for each of 480 candidates, totalling 2400 fits


{'alpha': 10.0,
 'fit_intercept': True,
 'max_iter': 2000,
 'positive': True,
 'selection': 'random',
 'tol': 0.1}

##### 3.5.2. Testing the newly acquired hyperparameters

In [16]:
lasso_model = Lasso(alpha=10.0, max_iter=2000, tol=0.1, positive=True, selection="random")

lasso_model.fit(X_train, y_train)

lasso_y_pred: np.ndarray = lasso_model.predict(X_train)
print_metrics(y_train, lasso_y_pred, "KNOWN")

lasso_y_pred: np.ndarray = lasso_model.predict(X_valid)
print_metrics(y_valid, lasso_y_pred, "NEW")

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 32413.460732008953
KNOWN DATA: RMSE: 140.1487714794666
KNOWN DATA: MAE: 114.47898814777542

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 32210.898324742975
NEW DATA: RMSE: 139.67687019521253
NEW DATA: MAE: 114.88469655511534



Performance is similar to all previous models.

##### Non-default hyperparameters:

- alpha (default=1.0): Constant that multiplies the L1 term, controlling regularization strength.
- max_iter (default=1000): The maximum number of iterations.
- tol (default=0.0001): The tolerance for the optimization: if the updates are smaller than tol, the optimization code checks the dual gap for optimality and continues until it is smaller than tol.
- positive (default=False): When set to True, forces the coefficients to be positive.
- selection (default='cyclic'): If set to ‘random’, a random coefficient is updated every iteration rather than looping over features sequentially by default.

[scikit-learn Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)

##### 3.6. Nearest Neighbour Regressor

In [17]:
from sklearn.neighbors import KNeighborsRegressor

# neighbour_model = KNeighborsRegressor(n_neighbors=25)
neighbour_model = KNeighborsRegressor()

neighbour_model.fit(X_train, y_train)

neighbour_y_pred: np.ndarray = neighbour_model.predict(X_train)
print_metrics(y_train, neighbour_y_pred, "KNOWN")

neighbour_y_pred: np.ndarray = neighbour_model.predict(X_valid)
print_metrics(y_valid, neighbour_y_pred, "NEW")

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 25548.988179546654
KNOWN DATA: RMSE: 124.1419373165785
KNOWN DATA: MAE: 99.74479885158914

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 43643.62094735951
NEW DATA: RMSE: 158.96222559059942
NEW DATA: MAE: 130.52321904371186



##### 3.6.1. Use GridSearch for attempt at optimal hyperparameters

In [18]:
params_neighbour = {
    "n_neighbors": [5, 10, 15, 20, 25, 50, 100],
    "weights": ["uniform", "distance"],
    "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
    "leaf_size": [30, 10, 5, 50, 100],
    "p": [2.0, 3.0, 5.0, 10.0, 1.0]
}

grid_neighbour = GridSearchCV(neighbour_model, param_grid=params_neighbour, n_jobs=-1, verbose=True)

grid_neighbour.fit(X_train, y_train)

grid_neighbour.best_params_

Fitting 5 folds for each of 1400 candidates, totalling 7000 fits


{'algorithm': 'auto',
 'leaf_size': 5,
 'n_neighbors': 100,
 'p': 1.0,
 'weights': 'uniform'}

##### 3.6.2. Testing the newly acquired hyperparameters

In [18]:
neighbour_model = KNeighborsRegressor(n_neighbors=100, leaf_size=5, p=1)

neighbour_model.fit(X_train, y_train)

neighbour_y_pred: np.ndarray = neighbour_model.predict(X_train)
print_metrics(y_train, neighbour_y_pred, "KNOWN")

neighbour_y_pred: np.ndarray = neighbour_model.predict(X_valid)
print_metrics(y_valid, neighbour_y_pred, "NEW")

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 31776.91288723784
KNOWN DATA: RMSE: 138.99791257912293
KNOWN DATA: MAE: 113.1102131266104

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 32269.61832787351
NEW DATA: RMSE: 139.62731346708705
NEW DATA: MAE: 113.90359437822303



##### Non-default hyperparameters:

- n_neighbours (default=5): Number of neighbors to use by default for kneighbors queries.
- leaf_size (default=30): Leaf size passed to BallTree or KDTree.
- p (default=2): Power parameter for the Minkowski metric.

[scikit-learn KNeighboursRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)

##### 3.7. Multi-Layer Perceptron (MLP) Regressor

In [19]:
from sklearn.neural_network import MLPRegressor

# mlp_model = MLPRegressor(solver="lbfgs")
mlp_model = MLPRegressor()

mlp_model.fit(X_train, y_train)

mlp_y_pred: np.ndarray = mlp_model.predict(X_train)
print_metrics(y_train, mlp_y_pred, "KNOWN")

mlp_y_pred: np.ndarray = mlp_model.predict(X_valid)
print_metrics(y_valid, mlp_y_pred, "NEW")

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 53933.93393078116
KNOWN DATA: RMSE: 175.4390867499404
KNOWN DATA: MAE: 140.68176707470596

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 48452.508672886695
NEW DATA: RMSE: 166.92525944248544
NEW DATA: MAE: 138.13820665249912



##### 3.7.1. Use GridSearch for attempt at optimal hyperparameters

In [20]:
params_mlp = {
    "hidden_layer_sizes": [100, 200, 400],
    "activation": ["identity", "logistic", "tanh", "relu"],
    "solver": ["lbfgs", "sgd", "adam"],
    "alpha": [0.0001, 0.001, 0.01, 0.1],
    "learning_rate": ["constant", "invscaling", "adaptive"],
    "tol": [0.0001, 0.001, 0.01, 0.1]
}

grid_mlp = GridSearchCV(mlp_model, param_grid=params_mlp, n_jobs=-1, verbose=True)

grid_mlp.fit(X_train, y_train)

grid_mlp.best_params_

Fitting 5 folds for each of 1728 candidates, totalling 8640 fits


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS R

{'activation': 'logistic',
 'alpha': 0.001,
 'hidden_layer_sizes': 400,
 'learning_rate': 'constant',
 'solver': 'sgd',
 'tol': 0.001}

##### 3.7.2. Testing the newly acquired hyperparameters

In [20]:
mlp_model = MLPRegressor(hidden_layer_sizes=400, activation="logistic", solver="sgd", alpha=0.001, tol=0.001)

mlp_model.fit(X_train, y_train)

mlp_y_pred: np.ndarray = mlp_model.predict(X_train)
print_metrics(y_train, mlp_y_pred, "KNOWN")

mlp_y_pred: np.ndarray = mlp_model.predict(X_valid)
print_metrics(y_valid, mlp_y_pred, "NEW")

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 32735.05607622639
KNOWN DATA: RMSE: 141.02315874228967
KNOWN DATA: MAE: 116.28102801125344

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 32393.560507128852
NEW DATA: RMSE: 141.3850490188678
NEW DATA: MAE: 116.59231658202586



With the last model we once again get the same results.

##### Non-default hyperparameters:

- hidden_layer_sizes (default=100): The ith element represents the number of neurons in the ith hidden layer.
- activation (default='identity'): Activation function for the hidden layer.
- solver (default="adam"): The solver for weight optimization.
- alpha (default=0.0001): Strength of the L2 regularization term. The L2 regularization term is divided by the sample size when added to the loss.
- tol (default=0.0001): Tolerance for the optimization.

[scikit-learn MLP Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html)

#### 5. Trying to predict for one ellipse all parameters individually

##### 5.1. Preparing data in the ellipse dataframe

In [75]:
shrunk_images_df: pd.DataFrame = pd.DataFrame(columns=["center_x", "center_y", "x_axis_length", "y_axis_length", "angle"])
for row in images_df.values:
    shrunk_images_df.loc[len(shrunk_images_df.index)] = [row[0], row[1], row[2], row[3], row[4]]
shrunk_images_df.index = images_df.index

shrunk_images_df

,center_x,center_y,x_axis_length,y_axis_length,angle
datetime,,,,,
2021-01-04,704.028564,442.499176,121.826691,451.231567,58.160427
2021-01-09,715.279602,351.457184,100.243324,367.353882,66.100555
2021-01-21,325.976746,490.977356,136.785980,500.891998,74.853523
2021-02-10,167.179871,545.132202,91.021637,191.275513,108.308769
2021-02-13,280.800171,504.206818,134.417603,405.863342,72.619293
...,...,...,...,...,...
2024-05-10,777.354004,107.631660,30.748890,90.560562,146.724838
2024-05-13,716.543701,305.779480,22.611246,165.610962,61.839939
2024-05-15,800.668274,145.144821,26.662622,81.719994,131.096573


In [76]:
center_x_image_df: pd.DataFrame = shrunk_images_df.drop(columns=["center_y", "x_axis_length", "y_axis_length", "angle"])
center_y_image_df: pd.DataFrame = shrunk_images_df.drop(columns=["center_x", "x_axis_length", "y_axis_length", "angle"])
x_axis_length_image_df: pd.DataFrame = shrunk_images_df.drop(columns=["center_x", "center_y", "y_axis_length", "angle"])
y_axis_length_image_df: pd.DataFrame = shrunk_images_df.drop(columns=["center_x", "center_y", "x_axis_length", "angle"])
angle_image_df: pd.DataFrame = shrunk_images_df.drop(columns=["center_x", "center_y", "x_axis_length", "y_axis_length"])

center_x_X_train, center_x_X_valid, center_x_Y_train, center_x_Y_valid = train_test_split(weather_df, center_x_image_df, test_size=0.1, random_state=69, shuffle=False)
center_y_X_train, center_y_X_valid, center_y_Y_train, center_y_Y_valid = train_test_split(weather_df, center_y_image_df, test_size=0.1, random_state=69, shuffle=False)
x_axis_length_X_train, x_axis_length_X_valid, x_axis_length_Y_train, x_axis_length_Y_valid = train_test_split(weather_df, x_axis_length_image_df, test_size=0.1, random_state=69, shuffle=False)
y_axis_length_X_train, y_axis_length_X_valid, y_axis_length_Y_train, y_axis_length_Y_valid = train_test_split(weather_df, y_axis_length_image_df, test_size=0.1, random_state=69, shuffle=False)
angle_X_train, angle_X_valid, angle_Y_train, angle_Y_valid = train_test_split(weather_df, angle_image_df, test_size=0.1, random_state=69, shuffle=False)

In [77]:
center_x_model = KNeighborsRegressor()
center_y_model = KNeighborsRegressor()
x_axis_length_model = KNeighborsRegressor()
y_axis_length_model = KNeighborsRegressor()
angle_model = KNeighborsRegressor()

center_x_model.fit(center_x_X_train, center_x_Y_train)
center_y_model.fit(center_y_X_train, center_y_Y_train)
x_axis_length_model.fit(x_axis_length_X_train, x_axis_length_Y_train)
y_axis_length_model.fit(y_axis_length_X_train, y_axis_length_Y_train)
angle_model.fit(angle_X_train, angle_Y_train)

KNeighborsRegressor()

In [78]:
print("\n===== CENTER X =====")
center_x_pred: np.ndarray = center_x_model.predict(center_x_X_train)
print_metrics(center_x_Y_train, center_x_pred, "KNOWN")

center_x_pred: np.ndarray = center_x_model.predict(center_x_X_valid)
print_metrics(center_x_Y_valid, center_x_pred, "NEW")

print("\n===== CENTER Y =====")
center_y_pred: np.ndarray = center_y_model.predict(center_y_X_train)
print_metrics(center_y_Y_train, center_y_pred, "KNOWN")

center_y_pred: np.ndarray = center_y_model.predict(center_y_X_valid)
print_metrics(center_y_Y_valid, center_y_pred, "NEW")


===== CENTER X =====
Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 50271.609594378126
KNOWN DATA: RMSE: 224.21331270550846
KNOWN DATA: MAE: 178.5388154830667

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 89894.7512630987
NEW DATA: RMSE: 299.82453412470886
NEW DATA: MAE: 243.2717088699341


===== CENTER Y =====
Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 15794.589611491572
KNOWN DATA: RMSE: 125.67652768712053
KNOWN DATA: MAE: 101.33612442282434

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 25738.3041532016
NEW DATA: RMSE: 160.43161830886578
NEW DATA: MAE: 142.45443429946897



In [79]:
print("\n===== X AXIS LENGTH =====")
x_axis_length_pred: np.ndarray = x_axis_length_model.predict(x_axis_length_X_train)
print_metrics(x_axis_length_Y_train, x_axis_length_pred, "KNOWN")

x_axis_length_pred: np.ndarray = x_axis_length_model.predict(x_axis_length_X_valid)
print_metrics(x_axis_length_Y_valid, x_axis_length_pred, "NEW")

print("\n===== Y AXIS LENGTH =====")
y_axis_length_pred: np.ndarray = y_axis_length_model.predict(y_axis_length_X_train)
print_metrics(y_axis_length_Y_train, y_axis_length_pred, "KNOWN")

y_axis_length_pred: np.ndarray = y_axis_length_model.predict(y_axis_length_X_valid)
print_metrics(y_axis_length_Y_valid, y_axis_length_pred, "NEW")


===== X AXIS LENGTH =====
Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 11164.249689829852
KNOWN DATA: RMSE: 105.66101310242038
KNOWN DATA: MAE: 82.34687781084705

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 12086.87113600489
NEW DATA: RMSE: 109.94030714894737
NEW DATA: MAE: 95.2607722759247


===== Y AXIS LENGTH =====
Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 36435.04528500789
KNOWN DATA: RMSE: 190.87966178985096
KNOWN DATA: MAE: 150.29870103776247

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 47526.547416673224
NEW DATA: RMSE: 218.00584262049773
NEW DATA: MAE: 190.51597973108292



In [80]:
print("\n===== ANGLE =====")
angle_pred: np.ndarray = angle_model.predict(angle_X_train)
print_metrics(angle_Y_train, angle_pred, "KNOWN")

angle_pred: np.ndarray = angle_model.predict(angle_X_valid)
print_metrics(angle_Y_valid, angle_pred, "NEW")


===== ANGLE =====
Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 1298.3228823371387
KNOWN DATA: RMSE: 36.0322478113306
KNOWN DATA: MAE: 29.033406945138857

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 2694.319077020119
NEW DATA: RMSE: 51.90683073565674
NEW DATA: MAE: 45.60538032054902



#### 5.2. Trying a GridSearch and applying this to all models

In [81]:
center_x_grid_model = KNeighborsRegressor()

params_center_x_grid = {
    "n_neighbors": [5, 10, 15, 20, 25, 50, 100],
    "weights": ["uniform", "distance"],
    "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
    "leaf_size": [30, 10, 5, 50, 100],
    "p": [2.0, 3.0, 5.0, 10.0, 1.0]
}

grid_center_x_grid = GridSearchCV(center_x_grid_model, param_grid=params_center_x_grid, n_jobs=-1, verbose=True)

grid_center_x_grid.fit(center_x_X_train, center_x_Y_train)

grid_center_x_grid.best_params_

Fitting 5 folds for each of 1400 candidates, totalling 7000 fits


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/lib/python3/dist-packages/scipy

{'algorithm': 'auto',
 'leaf_size': 30,
 'n_neighbors': 50,
 'p': 5.0,
 'weights': 'distance'}

In [82]:
center_y_grid_model = KNeighborsRegressor()

params_center_y_grid = {
    "n_neighbors": [5, 10, 15, 20, 25, 50, 100],
    "weights": ["uniform", "distance"],
    "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
    "leaf_size": [30, 10, 5, 50, 100],
    "p": [2.0, 3.0, 5.0, 10.0, 1.0]
}

grid_center_y_grid = GridSearchCV(center_y_grid_model, param_grid=params_center_y_grid, n_jobs=-1, verbose=True)

grid_center_y_grid.fit(center_y_X_train, center_y_Y_train)

grid_center_y_grid.best_params_

Fitting 5 folds for each of 1400 candidates, totalling 7000 fits


{'algorithm': 'brute',
 'leaf_size': 30,
 'n_neighbors': 25,
 'p': 1.0,
 'weights': 'uniform'}

In [83]:
x_axis_length_grid_model = KNeighborsRegressor()

params_x_axis_length_grid = {
    "n_neighbors": [5, 10, 15, 20, 25, 50, 100],
    "weights": ["uniform", "distance"],
    "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
    "leaf_size": [30, 10, 5, 50, 100],
    "p": [2.0, 3.0, 5.0, 10.0, 1.0]
}

grid_x_axis_length_grid = GridSearchCV(x_axis_length_grid_model, param_grid=params_x_axis_length_grid, n_jobs=-1, verbose=True)

grid_x_axis_length_grid.fit(x_axis_length_X_train, x_axis_length_Y_train)

grid_x_axis_length_grid.best_params_

Fitting 5 folds for each of 1400 candidates, totalling 7000 fits


{'algorithm': 'auto',
 'leaf_size': 30,
 'n_neighbors': 100,
 'p': 5.0,
 'weights': 'uniform'}

In [84]:
y_axis_length_grid_model = KNeighborsRegressor()

params_y_axis_length_grid = {
    "n_neighbors": [5, 10, 15, 20, 25, 50, 100],
    "weights": ["uniform", "distance"],
    "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
    "leaf_size": [30, 10, 5, 50, 100],
    "p": [2.0, 3.0, 5.0, 10.0, 1.0]
}

grid_y_axis_length_grid = GridSearchCV(y_axis_length_grid_model, param_grid=params_y_axis_length_grid, n_jobs=-1, verbose=True)

grid_y_axis_length_grid.fit(y_axis_length_X_train, y_axis_length_Y_train)

grid_y_axis_length_grid.best_params_

Fitting 5 folds for each of 1400 candidates, totalling 7000 fits


{'algorithm': 'auto',
 'leaf_size': 5,
 'n_neighbors': 100,
 'p': 1.0,
 'weights': 'uniform'}

In [85]:
angle_grid_model = KNeighborsRegressor()

params_angle_grid = {
    "n_neighbors": [5, 10, 15, 20, 25, 50, 100],
    "weights": ["uniform", "distance"],
    "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
    "leaf_size": [30, 10, 5, 50, 100],
    "p": [2.0, 3.0, 5.0, 10.0, 1.0]
}

grid_angle_grid = GridSearchCV(angle_grid_model, param_grid=params_angle_grid, n_jobs=-1, verbose=True)

grid_angle_grid.fit(angle_X_train, angle_Y_train)

grid_angle_grid.best_params_

Fitting 5 folds for each of 1400 candidates, totalling 7000 fits


{'algorithm': 'auto',
 'leaf_size': 100,
 'n_neighbors': 100,
 'p': 1.0,
 'weights': 'uniform'}

##### Applying this data to all models for hopelully improved performance

In [86]:
center_x_model = KNeighborsRegressor(n_neighbors=50, p=5, weights="distance")
center_y_model = KNeighborsRegressor(n_neighbors=25, p=1)
x_axis_length_model = KNeighborsRegressor(n_neighbors=100, p=5)
y_axis_length_model = KNeighborsRegressor(n_neighbors=100, leaf_size=5, p=1)
angle_model = KNeighborsRegressor(n_neighbors=100, leaf_size=100, p=1)

center_x_model.fit(center_x_X_train, center_x_Y_train)
center_y_model.fit(center_y_X_train, center_y_Y_train)
x_axis_length_model.fit(x_axis_length_X_train, x_axis_length_Y_train)
y_axis_length_model.fit(y_axis_length_X_train, y_axis_length_Y_train)
angle_model.fit(angle_X_train, angle_Y_train)

KNeighborsRegressor(leaf_size=100, n_neighbors=100, p=1)

In [87]:
print("\n===== CENTER X =====")
center_x_pred: np.ndarray = center_x_model.predict(center_x_X_train)
print_metrics(center_x_Y_train, center_x_pred, "KNOWN")

center_x_pred: np.ndarray = center_x_model.predict(center_x_X_valid)
print_metrics(center_x_Y_valid, center_x_pred, "NEW")

print("\n===== CENTER Y =====")
center_y_pred: np.ndarray = center_y_model.predict(center_y_X_train)
print_metrics(center_y_Y_train, center_y_pred, "KNOWN")

center_y_pred: np.ndarray = center_y_model.predict(center_y_X_valid)
print_metrics(center_y_Y_valid, center_y_pred, "NEW")


===== CENTER X =====
Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 0.0
KNOWN DATA: RMSE: 0.0
KNOWN DATA: MAE: 0.0

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 67722.05248619145
NEW DATA: RMSE: 260.23461046945977
NEW DATA: MAE: 203.62066064698755


===== CENTER Y =====
Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 19896.644837904976
KNOWN DATA: RMSE: 141.0554672386185
KNOWN DATA: MAE: 114.83658628244432

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 23171.47070405427
NEW DATA: RMSE: 152.22178130627125
NEW DATA: MAE: 133.52593096733094



In [88]:
print("\n===== X AXIS LENGTH =====")
x_axis_length_pred: np.ndarray = x_axis_length_model.predict(x_axis_length_X_train)
print_metrics(x_axis_length_Y_train, x_axis_length_pred, "KNOWN")

x_axis_length_pred: np.ndarray = x_axis_length_model.predict(x_axis_length_X_valid)
print_metrics(x_axis_length_Y_valid, x_axis_length_pred, "NEW")

print("\n===== Y AXIS LENGTH =====")
y_axis_length_pred: np.ndarray = y_axis_length_model.predict(y_axis_length_X_train)
print_metrics(y_axis_length_Y_train, y_axis_length_pred, "KNOWN")

y_axis_length_pred: np.ndarray = y_axis_length_model.predict(y_axis_length_X_valid)
print_metrics(y_axis_length_Y_valid, y_axis_length_pred, "NEW")


===== X AXIS LENGTH =====
Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 13903.642750278877
KNOWN DATA: RMSE: 117.91370891579518
KNOWN DATA: MAE: 93.48081843567226

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 12312.28446786848
NEW DATA: RMSE: 110.9607339010899
NEW DATA: MAE: 103.22777692005039


===== Y AXIS LENGTH =====
Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 45164.55540389187
KNOWN DATA: RMSE: 212.51954122831123
KNOWN DATA: MAE: 172.48795595969887

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 44481.61630552888
NEW DATA: RMSE: 210.9066530613221
NEW DATA: MAE: 184.23022470086812



In [89]:
print("\n===== ANGLE =====")
angle_pred: np.ndarray = angle_model.predict(angle_X_train)
print_metrics(angle_Y_train, angle_pred, "KNOWN")

angle_pred: np.ndarray = angle_model.predict(angle_X_valid)
print_metrics(angle_Y_valid, angle_pred, "NEW")


===== ANGLE =====
Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
KNOWN DATA: MSE: 1438.4790933168383
KNOWN DATA: RMSE: 37.92728692270037
KNOWN DATA: MAE: 31.435397416357382

Mean Squared Error, Root Mean Squared Error, Mean Absolute Error (perfect = 0.0)
NEW DATA: MSE: 2117.297074315452
NEW DATA: RMSE: 46.01409647396602
NEW DATA: MAE: 39.889330079406506

